# 6.14.2 Проект - решение


In [ ]:
import asyncio
import json
import re
import urllib.request
from typing import List, Dict, Any, Set

BANNED_URL = "https://raw.githubusercontent.com/nefelsay/asyncio/main/task_2_banned_word.txt"
MESSAGES_URL = "https://raw.githubusercontent.com/nefelsay/asyncio/main/task_2_message.json"

def load_banned_words() -> Set[str]:
    with urllib.request.urlopen(BANNED_URL, timeout=30) as resp:
        text = resp.read().decode("utf-8")
    return {line.strip().lower() for line in text.splitlines() if line.strip()}

def load_messages() -> List[Dict[str, Any]]:
    with urllib.request.urlopen(MESSAGES_URL, timeout=30) as resp:
        return json.loads(resp.read().decode("utf-8"))

def contains_banned(text: str, banned: Set[str]) -> bool:
    for w in banned:
        if re.search(rf"(?i)\b{re.escape(w)}\b", text):
            return True
    return False

def mask_banned(text: str, banned: Set[str]) -> str:
    for w in banned:
        text = re.sub(rf"(?i)\b{re.escape(w)}\b", "****", text)
    return text

async def handle_message(message: Dict[str, Any], banned: Set[str]) -> str:
    role = message.get("role")
    text = message.get("message", "")

    if role == "admin":
        return f"admin: {text}"

    if role == "moderator":
        return f"moderator: {mask_banned(text, banned)}"

    if role == "student":
        if contains_banned(text, banned):
            return "student: В сообщении есть запрещённое слово, сообщение скрыто"
        return f"student: {text}"

    if role == "black_list_user":
        return "black_list_user: Пользователь забанен, сообщение скрыто"

    if role is None:
        return "None: ERROR_USER_NONE"

    return f"{role}: {text}"

async def main():
    banned = load_banned_words()
    messages = load_messages()

    tasks = []
    for m in messages:
        role = m.get("role")
        tasks.append(asyncio.create_task(handle_message(m, banned), name=str(role)))

    results = await asyncio.gather(*tasks)

    for line in results:
        print(line)

asyncio.run(main())
